In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Radar_DataSet/data_SAAB_SIRS_77GHz_FMCW.npy"
MODEL_SAVE_PATH = "radar_cnn_lstm.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEQ_LEN = 5
BATCH_SIZE = 16
EPOCHS = 25
LR = 1e-3


In [3]:
data = np.load(DATA_PATH, allow_pickle=True)

print("Data type:", type(data))
print("Data shape:", data.shape)   # (130, 6)

df = pd.DataFrame(data)
df

Data type: <class 'numpy.ndarray'>
Data shape: (130, 6)


,0,1,2,3,4,5
0,[D1],"[[(-0.0803837295206183-0.024996274181930858j),...","[[62.26415142983489], [62.26415142983489], [62...","[[0.04242135490599999], [0.14313034690999998],...","[[2], [3], [1], [3], [1], [1], [1], [1], [3], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,[D1],"[[(0.1592469814114204-0.3764412575384074j), (0...","[[116.03773675560139], [116.03773675560139], [...","[[0.04531640558899999], [0.148743200275], [0.2...","[[3], [1], [3], [1], [1], [1], [2], [2], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
2,[D1],"[[(-0.36744600235662883-0.17434743507810269j),...","[[75.47169870283017], [75.47169870283017], [75...","[[0.04389842158099999], [0.14448924825099999],...","[[1], [1], [1], [1], [2], [1], [1], [3], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
3,[D1],"[[(-0.7918969406816826-0.0388248167751634j), (...","[[145.28302000294806], [145.28302000294806], [...","[[0.04372117357999999], [0.14372117358], [0.24...","[[1], [2], [3], [1], [3], [1], [3], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
4,[D1],"[[(0.08721619353018038-0.05631563733996364j), ...","[[22.641509610849056], [22.641509610849056], [...","[[0.03592226153599999], [0.13604042687], [0.23...","[[1], [1], [3], [1], [3], [1], [1], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
...,...,...,...,...,...,...
125,[CR],"[[(0.03368695343051483+0.37475747473197485j), ...","[[57.54717026090801], [57.54717026090801], [57...","[[0.145729984258], [0.24614356292700001], [0.3...","[[3], [1], [2], [3], [2], [1], [1], [3], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
126,[CR],"[[(-0.28511922708328735+0.32681276930952j), (-...","[[162.26415221108488], [162.26415221108488], [...","[[40.645375488256], [40.744902826920004], [40....","[[1], [1], [3], [3], [3], [1], [1], [2], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
127,[CR],"[[(0.2793143954436475-0.19863758289687442j), (...","[[178.30188818543627], [178.30188818543627], [...","[[39.94082612289701], [40.040944288231], [40.1...","[[1], [1], [2], [3], [3], [2], [3], [3], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
128,[CR],"[[(0.5004529387476391-0.4401659247452602j), (-...","[[192.45283169221693], [192.45283169221693], [...","[[39.952938069632005], [40.052938069632], [40....","[[2], [3], [1], [3], [1], [2], [3], [2], [2], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."


In [ ]:
DRONES = ['D1','D2','D3','D4','D5','D6']
BIRDS = [
    'seagull','pigeon','raven',
    'black-headed gull',
    'seagull and black-headed gull',
    'heron'
]

def get_label(label):
    if label in DRONES:
        return 1   
    elif label in BIRDS:
        return 0   
    else:
        return None


In [ ]:
class RadarSequenceDataset(Dataset):
    def __init__(self, npy_path, split=1):
        self.data = np.load(npy_path, allow_pickle=True)
        self.samples = []

        for row in self.data:
            label_str = row[0]
            radar_data = row[1]   
            split_info = row[4]

            label = get_label(label_str)
            if label is None:
                continue

            for i in range(radar_data.shape[1] - SEQ_LEN):
                if split_info[i] == split:
                    seq = radar_data[:, i:i+SEQ_LEN]
                    self.samples.append((seq, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq, label = self.samples[idx]
        frames = []

        for i in range(SEQ_LEN):
            frame = np.abs(seq[:, i])
            frame = frame.reshape(5, 256)
            frame = (frame - frame.min()) / (frame.max() - frame.min() + 1e-8)
            frames.append(frame)

        frames = np.array(frames, dtype=np.float32)
        x = torch.from_numpy(frames)
        y = torch.tensor(label, dtype=torch.long)

        return x, y



In [11]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d((1,2)),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((1,2)),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )

    def forward(self, x):
        x = self.net(x)
        return x.view(x.size(0), -1)


In [12]:
class RadarCNNLSTM(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = CNNFeatureExtractor()
        self.lstm = nn.LSTM(
            input_size=64,
            hidden_size=128,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        B, T, H, W = x.shape
        x = x.view(B*T, 1, H, W)

        features = self.cnn(x)
        features = features.view(B, T, -1)

        lstm_out, _ = self.lstm(features)
        out = lstm_out[:, -1, :]
        return self.fc(out)


In [13]:
train_ds = RadarSequenceDataset(DATA_PATH, split=1)
val_ds   = RadarSequenceDataset(DATA_PATH, split=2)
test_ds  = RadarSequenceDataset(DATA_PATH, split=3)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)

print(len(train_ds), len(val_ds), len(test_ds))


52157 6948 6955


In [ ]:
cuda = torch.device()
print(cuda)

In [14]:
model = RadarCNNLSTM().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/25 | Loss: 0.1429
Epoch 2/25 | Loss: 0.0667
Epoch 3/25 | Loss: 0.0497
Epoch 4/25 | Loss: 0.0401
Epoch 5/25 | Loss: 0.0344
Epoch 6/25 | Loss: 0.0317
Epoch 7/25 | Loss: 0.0291
Epoch 8/25 | Loss: 0.0251
Epoch 9/25 | Loss: 0.0227
Epoch 10/25 | Loss: 0.0205
Epoch 11/25 | Loss: 0.0191
Epoch 12/25 | Loss: 0.0166
Epoch 13/25 | Loss: 0.0147
Epoch 14/25 | Loss: 0.0138
Epoch 15/25 | Loss: 0.0130
Epoch 16/25 | Loss: 0.0106
Epoch 17/25 | Loss: 0.0102
Epoch 18/25 | Loss: 0.0092
Epoch 19/25 | Loss: 0.0085
Epoch 20/25 | Loss: 0.0077
Epoch 21/25 | Loss: 0.0067
Epoch 22/25 | Loss: 0.0071
Epoch 23/25 | Loss: 0.0054
Epoch 24/25 | Loss: 0.0052
Epoch 25/25 | Loss: 0.0051


In [15]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=1).cpu().numpy()

        y_true.extend(y.numpy())
        y_pred.extend(preds)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


Accuracy: 0.9955427749820274

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       971
           1       1.00      1.00      1.00      5984

    accuracy                           1.00      6955
   macro avg       0.99      1.00      0.99      6955
weighted avg       1.00      1.00      1.00      6955


Confusion Matrix:
 [[ 967    4]
 [  27 5957]]


In [16]:
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print("Model saved as:", MODEL_SAVE_PATH)

Model saved as: radar_cnn_lstm.pth


In [17]:
model = RadarCNNLSTM()
model.load_state_dict(torch.load("radar_cnn_lstm.pth", map_location="cpu"))
model.eval()


RadarCNNLSTM(
  (cnn): CNNFeatureExtractor(
    (net): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU()
      (10): AdaptiveAvgPool2d(output_size=(1, 1))
    )
  )
  (lstm): LSTM(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)